In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth==2025.7.10

In [ ]:
%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade transformers # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

In [ ]:
!pip install transformers -U

In [ ]:
!pip install --no-deps git+https://github.com/huggingface/transformers.git

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3n-E4B-it", 
    dtype = None, 
    max_seq_length = 8192,
    load_in_4bit = True, 
    full_finetuning = False, 
)

In [ ]:

from transformers import TextStreamer
def do_gemma_3n_inference(messages, max_new_tokens):
    _ = model.generate(
        **tokenizer.apply_chat_template(
            messages,
            add_generation_prompt = True, 
            tokenize = True,
            return_dict = True,
            return_tensors = "pt",
        ).to("cuda"),
        max_new_tokens = max_new_tokens,
        do_sample = False,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

In [ ]:
messages = [{
    "role": "user",
    "content": [{ "type" : "text",
                  "text" : """You are an assistant for a mental health company. Your task is to review the short note of a therapy session and generate case notes in first-person perspective, as if the therapist is personally writing them. Follow the specific template selected by the therapist for the session. Ensure the language used is professional, clear, and concise. Only include information explicitly mentioned in the audio, using direct quotes where appropriate to enhance accuracy. Avoid adding interpretations or assumptions beyond what was discussed in the session. Respond only with valid JSON. Do not write an introduction or summary.
        You should also add more details when creating the template.
        Here is a sample output for the SOAP template:
        {{
            "Subjective": generated notes here,
            "Objective": generated notes here,
            "Assessment": generated notes here,
            "Plan": generated notes here
        }}
        Here is the short note of the therapy session:  
        - Client discussed resurfacing memories from adolescence (specific content withheld); triggered by recent event in public space.
    - Emotional distress rated 8/10 during recall; tearful, avoided eye contact.
    - Expressed fear of being "damaged" and concerns about trust in close relationships.
    - Brief dissociative episode (~2 min); guided back w/ grounding (5-4-3-2-1).
    - Mentioned "recurring thoughts" of self-harm — no active plan or intent; safety plan reviewed.
    - Noted avoidance of specific place linked to prior trauma; client requested to explore this in future sessions.
    - Plan: reinforce coping skills, begin timeline work next week."""}]}]
do_gemma_3n_inference(messages, max_new_tokens = 2046)

In [ ]:
messages = [{
    "role": "user",
    "content": [{ "type" : "text",
                  "text" : """You are an assistant for a mental health company. Your task is to review the audio transcription of a therapy session and generate case notes in first-person perspective, as if the therapist is personally writing them. Follow the specific template selected by the therapist for the session. Ensure the language used is professional, clear, and concise. Only include information explicitly mentioned in the audio, using direct quotes where appropriate to enhance accuracy. Avoid adding interpretations or assumptions beyond what was discussed in the session. Respond only with valid JSON. Do not write an introduction or summary.
        Here is a sample output for the SOAP template:
        {{
            "Subjective": generated notes here,
            "Objective": generated notes here,
            "Assessment": generated notes here,
            "Plan": generated notes here
        }}
        Here is the transcription of the therapy session:  Hello Patrick, how are you today? It's good to see you. Let's begin where we left off in our last meeting. Okay. I've been thinking about it for a very long time. I was glad we got to speak about this before in our last sessions and you know I just I'll just find myself sitting there not even doing any work I'll just be thinking about this promotion and you know how I've been there for a long time and nothing seems to be you know I just feel like I'm going to be passed up for a promotion every time one comes up you know I work with you know some very nice people but they don't work as hard as I do. I put my all into my work every day but that's just how I do and maybe that's a fault of mine. Maybe I shouldn't put so much of myself into work. But there I am sitting at my desk watching my Netflix sometimes on my phone. really want this promotion and I know I do work hard when I'm not worth watching the Netflix and you know I crunch numbers all day it's something that I've liked and I went to school to learn and I really enjoy it but you know if I keep getting passed up from promotions you know, where does that leave me in the company? I feel like, you know, I put in overtime when I'm asked to. I, you know, will come in on weekends even though I don't want to work on weekends, but there I am. So you feel like you're working hard but they're not noticing yeah yeah um and you know some of my co-workers seem to uh notice that that the work that i put in for extra overtime and whatnot but um you know um they uh and i know they're working hard too but i'm telling you it's just really stresses me out and sometimes I just need a release you know to think about work so much and so even when I go home to my family that I will I find that I'll bring my stress home to work and I'll yell at my wife or I'll tell my daughter to go, go clean your room. And that's just, that's just not me. I don't yell. But when I think about work, when I come home, that's just, I don't want to bring work home with me. I just want to come home, relax, and enjoy the rest of my evening. And then do the same routine the next day. Get up, go to work, do the best I can. So your work stresses are coming home with you and that bothers you? Yes, it hadn't always bothered me. You know, I um, I um, I'll come home and my wife will want me to cook after being at work for eight hours. My kid will want me to help her with her coloring books. I don't like to color, but I'll help her. Anyhow, and I, and I don't want to sound like my family's a burden, because they're not. They're, um, they're really great people too, and I love them, but that's just what the work stressors come and do for me, you know, and so I'm trying to deal with work and home and, you know, I don't want it to affect either. I don't want work to affect my home life and vice versa. I don't want my home life to affect, you know, what goes on at work. I want to be concentrated and focused on work, but, you know, maybe I just need some more exercises you know to take a break and find new ways to push that energy that anxiety for not getting the promotion you know because I see there people that have been there for less time than I have been there and they get the promotion that I should be the one that get the promotion do you think I'm a bad person I hear the frustration in your voice and I hear you saying that you feel like you work hard and that you deserve the promotion that is being offered. Yes, I'm so glad you're hearing me. My wife, she doesn't understand this. I'll tell her and it's like it goes in one ear and out the other. And then my kid with her coloring books, she can't help me. But you know, it's good to be able to have a place to where I can come and release this out. And I don't mean to scare you or, you know, I'm not a violent person, mostly. But you know, I want everything be I wanted I want that promotion I've earned it those others haven't earned it sorry see this is this is what I didn't want. I don't like it affecting my life this way. It's just one silly promotion. But it affects me so hard. But, you know, I mostly just fight it back down. And these sessions are really helping. I got to admit, you know, like I said, I mostly keep it trapped in. But, you know, if you had any advice for me, you me, I would appreciate it. But it just feels like it's overwhelming and kind of hard dealing with work sometimes. I have a picture of my family, and that helps calm me down at work. Sometimes I'll go into the bathroom and just punch the wall, you know, just hit the wall, get my frustration, then I'm fine. But anyhow, these sessions are really helping and I appreciate you listening and you know I look forward to these sessions and I know we'll have another one at some point and I appreciate your advice you know whatever you can give me to help me through this what is it whatever you can give me to help me through this. What is it that you would want me to say to you? I guess it's just the encouragement I don't get from my wife. When she tells me I don't do a good job. But, you know, that's not your position. That's something me and her have to work over. And my kid's coloring books, it's fine. She colors a lot. That's what she does. But I appreciate our time and I can't wait to go to our you know our next session okay well that sounds like a good stopping point right there and so we'll meet again at the same time next week yes I'll be able to make that appointment and I'll take the advice that you've given me and you know use that constructively at my job""" }]
}]
do_gemma_3n_inference(messages, max_new_tokens = 1024)

In [ ]:
messages = [{
    "role": "user",
    "content": [{ "type" : "text",
                  "text" : """You are an assistant for a mental health company. Your task is to review the short note of a therapy session and generate case notes in first-person perspective, as if the therapist is personally writing them. Follow the specific template selected by the therapist for the session. Ensure the language used is professional, clear, and concise. Only include information explicitly mentioned in the audio, using direct quotes where appropriate to enhance accuracy. Avoid adding interpretations or assumptions beyond what was discussed in the session. Respond only with valid JSON. Do not write an introduction or summary.
        Here is a sample output for the SOAP template:
        {{
            "Subjective": generated notes here,
            "Objective": generated notes here,
            "Assessment": generated notes here,
            "Plan": generated notes here
        }}
        Here is the short note of the therapy session: Mood: Anxious, 3/10\nDiscussed: financial worries, academic pressure\nProgress: increased motivation, better concentration\nChallenges: negative self‑talk, relapse triggers\nStrategies: journaling, behavioral activation\nGoals: daily mindfulness, set boundaries with family""" }]
}]
do_gemma_3n_inference(messages, max_new_tokens = 1024)

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, 
    finetune_language_layers   = True,  
    finetune_attention_modules = True,  
    finetune_mlp_modules       = True, 
    r = 8,          
    lora_alpha = 8,  
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
from datasets import load_dataset

url = "/kaggle/input/therapist-notes-v2/gemma_therapy_dataset_10k.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train")

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [ ]:
dataset[100]

In [ ]:
def formatting_prompts_func(examples):
   convos = examples["messages"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

<a name="Train"></a>
### Train the model

In [ ]:
!nvidia-smi

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, 
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Model Training

In [ ]:
trainer_stats = trainer.train()

In [ ]:
gc.collect()
torch.cuda.empty_cache()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{ "type" : "text",
                  "text" : """You are an assistant for a mental health company. Your task is to review the short note of a therapy session and generate case notes in first-person perspective, as if the therapist is personally writing them. Follow the specific template selected by the therapist for the session. Ensure the language used is professional, clear, and concise. Only include information explicitly mentioned in the audio, using direct quotes where appropriate to enhance accuracy. Avoid adding interpretations or assumptions beyond what was discussed in the session. Respond only with valid JSON. Do not write an introduction or summary.
        Here is a sample output for the SOAP template:
        {{
            "Subjective": generated notes here,
            "Objective": generated notes here,
            "Assessment": generated notes here,
            "Plan": generated notes here
        }}
        Here is the short note of the therapy session: Mood: Anxious, 3/10\nDiscussed: financial worries, academic pressure\nProgress: increased motivation, better concentration\nChallenges: negative self‑talk, relapse triggers\nStrategies: journaling, behavioral activation\nGoals: daily mindfulness, set boundaries with family""" }]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, 
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens = 1024, 
    temperature = 1.0, top_p = 0.5, top_k = 64,
)
tokenizer.batch_decode(outputs)

<a name="Save"></a>
### Saving, loading finetuned models

In [ ]:
model.push_to_hub_merged(
"vannotanno/gemma3n-finetune-notes-gguf", tokenizer
)

In [ ]:
model.save_pretrained_gguf(
    "/root/.cache/gemma_3n_finetuned",
    quantization_type = "Q8_0",
)

In [ ]:
model.push_to_hub_gguf(
    "/root/.cache/gemma_3n_finetuned",
    quantization_type = "Q8_0",
    repo_id = "vannotanno/gemma3n-finetune-notes-gguf",
)